# Setup

In [1]:
using ForwardDiff

# Using ForwardDiff to automatically differentiate functions

## Jacobians

Let's define a linear function:

In [2]:
const A = [1. 2.;
           2. 3.]
flinear(x) = A * x

flinear (generic function with 1 method)

ForwardDiff allows you to compute the Jacobian of this function *at a specific value of the arguments*:

In [3]:
ForwardDiff.jacobian(flinear, zeros(2))

2×2 Array{Float64,2}:
 1.0  2.0
 2.0  3.0

And here's a nonlinear function:

In [4]:
fnonlinear(x) = A * sqrt.(x) + 4 * x.^3

fnonlinear (generic function with 1 method)

In [5]:
ForwardDiff.jacobian(fnonlinear, [1., 2.])

2×2 Array{Float64,2}:
 12.5   0.707107
  1.0  49.0607  

## Hessians

Here's a quadratic form:

In [6]:
fquadratic(x) = 1/2 * dot(x, A * x)

fquadratic (generic function with 1 method)

ForwardDiff allows you to compute the Hessian automatically:

In [7]:
ForwardDiff.hessian(fquadratic, zeros(2))

2×2 Array{Float64,2}:
 1.0  2.0
 2.0  3.0

## How does it work?

ForwardDiff implements forward-mode [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation).

Automatic differentiation is **not**:
* numeric differentiation
* symbolic differentiation

Automatic differentiation tools compute derivatives **without numerical approximation**, but **only at a specific point**.

They do so by:
* implementing a new 'scalar' type (similar to the standard `Float64`), and 
* overloading basic methods to implement the elementary rules of differentiation.

`Dual` is ForwardDiff's automatic differentiation scalar type:

In [11]:
dump(ForwardDiff.Dual)

ForwardDiff.Dual{N,T<:Real} <: Real
  value::T
  partials::ForwardDiff.Partials{N,T}


Let's create some `ForwardDiff.Dual`s:

In [17]:
import ForwardDiff: Dual, Partials
x = Dual(1., Partials((1., 0.)))
y = Dual(2., Partials((0., 2.)))

Dual(2.0,0.0,2.0)

and add them together:

In [21]:
x + y

Dual(3.0,1.0,2.0)

In [22]:
@which x + y

+{N}(n1::ForwardDiff.Dual{N,T<:Real}, n2::ForwardDiff.Dual{N,T<:Real}) at /Users/twan/code/julia/robocon2017/v0.5/ForwardDiff/src/dual.jl:213

Here's another example:

In [28]:
exp(x)

Dual(2.718281828459045,2.718281828459045,0.0)

In [29]:
@which exp(x)

exp{N}(n::ForwardDiff.Dual{N,T<:Real}) at /Users/twan/code/julia/robocon2017/v0.5/ForwardDiff/src/dual.jl:341

Implements:

$$
\frac{\partial e^{x(z)}}{\partial z} = \frac{\partial e^{x}}{\partial x} \frac{\partial x}{\partial z} = e^x \frac{\partial x}{\partial z}
$$